In [2]:
import pandas as pd

import nltk
from gensim.models import Word2Vec 

import requests
import re

In [78]:
text_url = 'http://www.gutenberg.org/files/2554/2554-0.txt'
text = requests.get(text_url).text

In [79]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

def cleanRawText(text):
    #To lower
    text = text.lower()
    #Removing Unicode Characters
    text = text.encode('ascii', 'ignore').decode()
    #Removing Links
    text = re.sub("https*\S+", "", text)
    text = re.sub("www.*\S+", "", text)
    #Removing Ticks and the Next Character
    text = re.sub("\'\w+", '', text)
    #Removing Punctuation except '.'
    text = re.sub("[^\w\s]", '', text)
    #Removing Numbers and Extra Lines
    text = re.sub("[\r\n\d\_]", ' ', text)
    #Removing Extra Spaces
    text = re.sub(" +", ' ', text)
    
    #Removing StopWords
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

In [80]:
text = cleanRawText(text)

In [81]:
text = text.split(' ')

In [82]:
model1 = Word2Vec([text], min_count=1, size=300, workers=4, window=10, iter=10)

In [83]:
print(model1)

Word2Vec(vocab=10447, size=300, alpha=0.025)


In [84]:
list(model1.wv.vocab)[:20]

['project',
 'gutenberg',
 'ebook',
 'crime',
 'punishment',
 'fyodor',
 'dostoevsky',
 'use',
 'anyone',
 'anywhere',
 'cost',
 'almost',
 'restrictions',
 'whatsoever',
 'may',
 'copy',
 'give',
 'away',
 'reuse',
 'terms']

In [85]:
model1.most_similar('man')

C:\Users\Gray\Anaconda3\envs\datascience\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('dounia', 0.9983670711517334),
 ('even', 0.998171865940094),
 ('one', 0.9981217980384827),
 ('would', 0.9980551600456238),
 ('though', 0.9980179071426392),
 ('room', 0.9979327321052551),
 ('upon', 0.9978546500205994),
 ('first', 0.9977613687515259),
 ('us', 0.9977599382400513),
 ('time', 0.9976540207862854)]

In [86]:
df = pd.read_csv('Online Retail.csv')

In [87]:
df = df.dropna()
df_text = list(df.Description)

In [88]:
df_text = ' '.join(df_text)

In [90]:
df_text = cleanRawText(df_text)
df_text = df_text.split(' ')

In [91]:
model2 = Word2Vec([df_text], min_count=1, size=300, workers=4, window=10, iter=10)

In [92]:
print(model2)

Word2Vec(vocab=2047, size=300, alpha=0.025)


In [ ]:
model2.wv.vocab